# Plot TPC-H results

This currently assumes that benchmarks have been run and have populated benchmark.db.  It also assumes that that database has only those results and from only one run (this is usually a bad assumption).

```
rm benchmark.db
pytest --benchmark tests/tpch
```

In [ ]:
import pandas as pd

df = pd.read_sql_table(table_name="test_run", con="sqlite:///../../benchmark.db")

df = df[
    (df.call_outcome == "passed")
    & (df.path.str.contains("^tpch/test_(?:dask|duckdb|polars|pyspark)"))
    & df.cluster_name
]
df = df[["path", "name", "duration", "start", "cluster_name"]]

df["library"] = df.path.map(lambda path: path.split("_")[-1].split(".")[0])
df["query"] = df.name.map(lambda name: int(name.split("_")[-1]))
df["name"] = df.cluster_name.map(lambda name: name.split("-", 3)[-1])
df["scale"] = df.cluster_name.map(lambda name: int(name.split("-")[2]))
del df["path"]
del df["cluster_name"]

In [ ]:
df

In [ ]:
df = df.sort_values(["query", "library"])

def recent(df):
    return df.sort_values("start").iloc[-1]

df = df.groupby(["library", "query"]).apply(recent).reset_index(drop=True)
del df["start"]
df.head(10)

In [ ]:
import altair as alt

chart = alt.Chart(df).mark_bar().encode(
    x="query:N",
    y="duration:Q",
    xOffset="library:N",
    color=alt.Color('library').scale(
        domain=["dask", "duckdb", "polars", "pyspark"], 
        range=["#5677a4", "#e68b39", "#d4605b", "green"],
    ),
    tooltip=["library", "duration"]
).properties(
    title=f"TPC-H -- scale:{df.scale.iloc[0]} name:{df.name.iloc[0]}"
).configure_title(
    fontSize=20,

)
chart